<a href="https://colab.research.google.com/github/VincentZuo/fin-models/blob/main/%5BFinal%5D_Failed_v10_torch_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up and read data

In [ ]:
!pip install transformers==4.17.0
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.8 MB 4.3 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 880 kB 63.4 MB/s 
     |████████████████████████████████| 6.6 MB 63.6 MB/s 
     |████████████████████████████████| 596 kB 42.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=1fba14a8fb64328efb82c239ea8088f8082f65c86b435b7ee43fd025fc9e0f2d
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 3.0 MB/s 
     |███████

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
import torch
import transformers
print(transformers.__version__)
print(torch.__version__)

4.17.0
1.11.0+cu113


In [ ]:
import sys
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
read_df = pd.read_pickle("/content/gdrive/MyDrive/224U_2022/final_project_data.pkl")

In [ ]:
read_df.head()

,title,label,range,emb
0,Joseph Greenberg,"[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","(0, 106)","[[-0.07557425, -0.011484241, -0.07807306, 0.00..."
1,Pauline Donalda,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","(106, 127)","[[-0.057122264, 0.005889078, -0.030671664, -0...."
2,List of German football transfers summer 2017,"[0, 0, 1, 0, 0]","(127, 132)","[[-0.0032938644, -0.02310217, -0.05397396, -0...."
3,Lester Hudson,"[0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, ...","(132, 176)","[[-0.0091192825, 0.090078354, -0.029187214, -0..."
4,Monique Ganderton,"[1, 0, 0, 0, 0, 1, 0]","(176, 183)","[[-0.090651, -0.09176681, -0.029592248, -0.062..."


In [ ]:
MAX_SEQ_LENGTH = 128
EMB_SIZE_SENTS = 384

In [ ]:
from sentence_transformers import SentenceTransformer
#Define the model
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
empty_sentence = sentence_model.encode([''])
MAX_SEQ_LENGTH=128
FAKE_INDEX = 0

In [ ]:
def dummy_emb():
  return empty_sentence[0]

def pad_emb(current_emb, seq_len):
  if len(current_emb) > seq_len:
    return current_emb[0:seq_len]
  padding =  np.array([dummy_emb()] * (seq_len - len(current_emb)), dtype = 'double')
  return np.concatenate([current_emb, padding], axis=0)

def pad_label_logits(current_label, seq_len):
  def sublist_cr(index_sub):
    sublist = [0, 0, 0]
    sublist[int(index_sub)] = 1
    return sublist
  new_current = [sublist_cr(item) for item in current_label]
  if len(new_current) > seq_len:
    return new_current[0:seq_len]
  padding =  [sublist_cr(2)] * (seq_len - len(new_current))
  return np.concatenate([new_current, padding], axis=0)

In [ ]:
pad_label_logits([1,0,1], 6)

array([[0, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

In [ ]:
from torch.utils.data import Dataset

class WikiPandasDfDataSet(Dataset):
  def __init__(self, input_df):
    self.pandas_df = input_df

  def __getitem__(self, index):
    seq = np.array(self.pandas_df.iloc[index]["emb"], dtype='double')
    label = np.array(self.pandas_df.iloc[index]["label"], dtype='double')
    return seq, label

  def __len__(self):
      return len(self.pandas_df)

def collate_fn(batch):
    batched_seq = []
    batched_label = []
    for seqs, labels in batch:
      try:
        local_label = pad_label_logits(labels, MAX_SEQ_LENGTH)
        local_emb = pad_emb(seqs, MAX_SEQ_LENGTH)
        batched_seq.append(local_emb)
        batched_label.append(local_label)
      except Exception as e:
        print("collate_fn err:", e)
        continue
    return torch.tensor(batched_seq).float(), torch.tensor(batched_label).float()

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE_TRAIN = 32

dataset_train = WikiPandasDfDataSet(read_df.head(3200))
train_dataloader = DataLoader(dataset_train, batch_size=BATCH_SIZE_TRAIN, collate_fn = collate_fn, shuffle=True, drop_last=True)
dataset_val = WikiPandasDfDataSet(read_df.tail(320))
val_dataloader = DataLoader(dataset_val, batch_size=BATCH_SIZE_TRAIN, collate_fn = collate_fn, shuffle=False, drop_last=True)

# NN Model code
https://pytorch.org/tutorials/beginner/translation_transformer.html

In [ ]:
import torch.nn as nn
import datetime
import math
from torch.nn import TransformerEncoderLayer

In [ ]:
class LstmGenerator(nn.Module):
  def __init__(self, seq_length=128, emb_dim=384):
      super(LstmGenerator, self).__init__()
      self.lstm = nn.LSTM(emb_dim, self.hidden_dim, batch_first=True, bidirectional=True)
  def forward(self, noise):
      return self.lstm(noise)

class MlpGenerator(nn.Module):
    def __init__(self, noise_length, seq_length, hidden_sizes=[512], dropout_rate=0.1):
        super(MlpGenerator, self).__init__()
        self.seq_length = seq_length
        self.emb_dim = emb_dim
        layers = []
        hidden_sizes = [noise_length] + hidden_sizes + [emb_dim * seq_length]
        for i in range(len(hidden_sizes)-1):
            layers.extend([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]), nn.LeakyReLU(0.2, inplace=True), nn.Dropout(dropout_rate)])
        self.layers = nn.Sequential(*layers)

    def forward(self, noise):
        output_rep = self.layers(noise)
        return torch.reshape(output_rep, [output_rep.shape[0], self.seq_length, self.emb_dim])

class BiLstmDiscriminator(nn.Module):
  def __init__(self, hidden_dim=512, emb_dim=384):
    super().__init__()
    self.hidden_dim = hidden_dim
    self.lstm = nn.LSTM(emb_dim, self.hidden_dim, batch_first=True, bidirectional=True)
    # Dim 1 is for break/no-break, dim 2 is for real/fake
    self.classifier_layer = nn.Linear(self.hidden_dim*2, 4)
    self.activation = nn.Softmax(dim=-1)

  def forward(self, seq_input):
    seq_features, pooled = self.lstm(seq_input)
    linear_output = self.classifier_layer(seq_features)
    return seq_features, linear_output, self.activation(linear_output)

In [ ]:
import torch.nn.functional as F

DROP_PROB = 0.6

def random_sel_mask(batch_size, prob=DROP_PROB):
  res = np.random.choice(2, batch_size, p=[1-prob, prob])
  return torch.tensor(res).bool()

In [ ]:
a = torch.tensor([[[1], [0.5]], [[0.1], [0.3]]])
b = torch.tensor([[[1], [0.5]], [[0.4], [0.6]]])

In [ ]:
torch.sum(torch.tensor(random_sel_mask(10, prob=DROP_PROB)).int())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


tensor(5)

# Training loop code

In [ ]:
from tqdm import tqdm
from transformers import AdamW
from sklearn.metrics import f1_score,accuracy_score

In [ ]:
NUM_EPOCS = 3
NOISE_LEN = 512

generator_ = MlpGenerator(noise_length=NOISE_LEN, seq_length=MAX_SEQ_LENGTH)
discriminator_ = BiLstmDiscriminator()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator_.to(device)
discriminator_.to(device)

gen_optimizer = AdamW(generator_.parameters(),lr = 0.05)
dis_optimizer = AdamW(discriminator_.parameters(),lr = 0.05)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
def train(train_dataloader, epsilon=1e-8):

  g_loss_train, d_loss_train = 0, 0
  step = 0
  generator_.train()        
  discriminator_.train()   

  bce_criterion = nn.BCELoss(reduction='none')  
  # iterate over batches
  for batch in tqdm(train_dataloader):
      print(step)
      sequences, labels = batch[0].to(device), batch[1].to(device) 
      batch_size = sequences.shape[0]
      noise = torch.zeros(sequences.shape[0], NOISE_LEN, device=device).uniform_(0, 1)
      gen_seqs = generator_(noise)

      # disciminator_input = torch.cat([sequences, gen_seqs], dim=0)
      real_f, real_logits, real_p = discriminator_(sequences)
      fake_f, fake_logits, fake_p = discriminator_(gen_seqs)


      g_loss_d = -1 * torch.mean(torch.log(1 - fake_p[:,-1] + epsilon))
      g_feat_reg = torch.mean(torch.pow(torch.mean(real_f, dim=0) - torch.mean(fake_f, dim=0), 2))
      g_loss = g_loss_d + g_feat_reg

      label_mask = random_sel_mask(real_f.shape[0]).to(device)
      final_logits = real_logits[:,:, 0:-1]
      log_probs = F.log_softmax(final_logits, dim=-1)
      product = torch.mean(labels * log_probs, dim=-1)
      # print("log_probs:", log_probs.shape, log_probs)
      # print("Product:", product.shape, product)
      per_example_loss = -torch.mean(product, dim=-1)
      per_example_loss = -torch.masked_select(per_example_loss, label_mask.to(device))
      labeled_example_count = per_example_loss.type(torch.float32).numel()

      # It may be the case that a batch does not contain labeled examples, 
      # so the "supervised loss" in this case is not evaluated
      if labeled_example_count == 0:
        D_L_Supervised = 0
      else:
        D_L_Supervised = torch.div(torch.sum(per_example_loss.to(device)), labeled_example_count)

      D_L_unsupervised1U = -1 * torch.mean(torch.log(1 - real_p[:,:,-1] + epsilon))
      D_L_unsupervised2U = -1 * torch.mean(torch.log(fake_p[:,:,-1] + epsilon))
      d_loss = D_L_Supervised + D_L_unsupervised1U + D_L_unsupervised2U

      # Avoid gradient accumulation
      gen_optimizer.zero_grad()
      dis_optimizer.zero_grad()

      g_loss.backward(retain_graph=True)
      d_loss.backward() 
      
      torch.nn.utils.clip_grad_norm_(generator_.parameters(), 1.0)
      torch.nn.utils.clip_grad_norm_(discriminator_.parameters(), 1.0)

      # Apply modifications
      gen_optimizer.step()
      dis_optimizer.step()
      
      g_loss_train += g_loss.item()
      d_loss_train += d_loss.item()

      if step % 5 == 0 and not step == 0:
        print('Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))
        print("loss",[g_loss.item(), d_loss.item(), D_L_Supervised.item(), D_L_unsupervised1U.item(), D_L_unsupervised2U.item()])
      step+=1
  avg_train_loss_g = g_loss_train / len(train_dataloader)
  avg_train_loss_d = d_loss_train / len(train_dataloader)    
  return avg_train_loss_g, avg_train_loss_d

In [ ]:
def eval(test_dataloader):
  discriminator_.eval()
  generator_.eval()

  # Tracking variables 
  total_test_accuracy = 0

  total_test_loss = 0
  nb_test_steps = 0

  all_preds = []
  all_labels = []

  nll_loss = torch.nn.CrossEntropyLoss(ignore_index=-1)
  for batch in test_dataloader:
      sequences, eval_labels = batch[0].to(device), batch[1].to(device)       
      # Tell pytorch not to bother with constructing the compute graph during
      # the forward pass, since this is only needed for backprop (training).
      with torch.no_grad():        
          _, _, real_towers = discriminator_(sequences)
          real_pred_t = real_towers[:,:,0:3]
          ts= real_pred_t.shape
          preds = torch.reshape(real_pred_t, [ts[0], ts[1], 3])
          loss = nll_loss(eval_labels, preds)
          print("Test loss:", loss.item())
          
      # Accumulate the predictions and the input labels
      all_preds += preds.detach().cpu().numpy().flatten().tolist()
      all_labels += eval_labels.detach().cpu().numpy().flatten().tolist()

  # Report the final accuracy for this validation run.
  avg_test_loss = total_test_loss / len(test_dataloader)
  return avg_test_loss, all_preds, all_labels

In [ ]:
train_losses = []
valid_losses = []
accuracy_list = []
f1_list = []

for epoch in range(NUM_EPOCS):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, NUM_EPOCS))
    
    #train model
    avg_train_loss_g, avg_train_loss_d = train(train_dataloader)

    valid_loss, e_pred, e_label = eval(val_dataloader)
    # for threshold in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]:
    #   labels_bool = [ elem > 0 for elem in e_label]
    #   preds_bool = [elem > threshold for elem in e_pred]
    #   accuracy = accuracy_score(labels_bool, preds_bool)
    #   valid_f1 = f1_score(labels_bool, preds_bool,average="macro")
    #   print('Threshold:', threshold,'Accuracy:', accuracy, 'F1:', valid_f1)



 Epoch 1 / 3


  0%|          | 0/100 [00:00<?, ?it/s]

collate_fn err: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)
0


  1%|          | 1/100 [00:00<00:51,  1.92it/s]

collate_fn err: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)
1


...

collate_fn err: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)
98


 99%|█████████▉| 99/100 [00:49<00:00,  1.99it/s]

99


100%|██████████| 100/100 [00:50<00:00,  1.98it/s]


Test loss: 208.63156127929688
Test loss: 208.85617065429688
Test loss: 208.67013549804688
collate_fn err: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)
collate_fn err: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)
Test loss: 208.92591857910156
Test loss: 209.0422821044922
collate_fn err: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)
collate_fn err: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)
Test loss: 208.96719360351562
Test loss: 209.060546875
Test loss: 208.41354370117188
collate_fn err: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) 

In [ ]:
torch.